In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from astropy.io import fits
%matplotlib inline

In [ ]:
#Part 1

#We will use the coordinates of the star as a first guess for its pixel location.
def radecsex(h, m, s, D, M, S):
    first = h
    second = m / 60.
    third = s / 3600.
    newra = 15.0 * (first + second + third)
    
    sign = np.sign(D)
    FIRST = np.abs(D)
    SECOND = M / 60.
    THIRD = S / 3600.
    newdec = sign * (FIRST + SECOND + THIRD)
    
    return newra, newdec

coords_42b = radecsex(16., 31., 15.016, -24., 32., 43.70)
print(coords_42b)

coords_12 = radecsex(16., 26., 28.039, -25., 26., 47.80)
print(coords_s12)


In [ ]:
#Part 2

ROXs42b = 'KOA_2439/NIRC2/calibrated/'
ROXs12 = 'KOA_3783/NIRC2/calibrated/'
filenames_42b = np.genfromtxt(ROXs42b + 'paramfile2.dat', dtype='str')
filenames_12 = np.genfromtxt(ROXs12 + 'paramfile2.dat', dtype='str')
#filenames_42b = longfilenames_42b.split('_')[0]

#It looks like there's an RA & dec offset of 0.2" so we're just going to force it into the code.

count = 0
def shift(directory, filenames, coords, shiftx, shifty):
    for f in filenames:
        ra, dec = coords
        file = fits.open(directory + f)
        pixelscale = file[0].header['PIXSCALE']/3600. #it's given in arcsec/pixel, convert to deg
        print(pixelscale)
        pix_ref_ra = file[0].header['CRPIX1']
        pix_ref_dec = file[0].header['CRPIX2']
        coord_ref_ra = file[0].header['CRVAL1']
        coord_ref_dec = file[0].header['CRVAL2']
        flux = file[0].data

        #input will be ra, dec
        star_location_x = -(ra+(shiftx/3600.)-coord_ref_ra)/pixelscale + pix_ref_ra
        star_location_y = (dec-(shifty/3600.)-coord_ref_dec)/pixelscale + pix_ref_dec

        #define a mask to lay down
        #15 pixel radius should be fine, but be careful to cut out where light leaks from coronagraph.
        x_hi = int(round(star_location_x + 15))
        x_low = int(round(star_location_x - 15))
        y_hi = int(round(star_location_y + 15))
        y_low = int(round(star_location_y - 15))

        #We will choose an arbitrary flux threshold of 3,000 to exclude from the region.
        for x in range(x_low, x_hi):
            for y in range(y_low, y_hi):
                if flux[x, y] > 3000.:
                    flux[x, y] = flux[x, y] - 3000.

        x = range(x_low, x_hi)
        y = range(y_low, y_hi)

        (X,Y) = np.meshgrid(x,y)

        x_coord = (X*flux[X, Y]).sum() / flux[X, Y].sum()
        y_coord = (Y*flux[X, Y]).sum() / flux[X, Y].sum()
        
        print(x_coord, y_coord)

        #centroid = np.array([int(x_coord), int(y_coord)])
        
        newfilename = f.split("_")[0]
        np.savetxt(directory + str(newfilename) + '.txt', [f, x_coord, y_coord], fmt='%s')
        
        #count +=1
        
#a1 = shift(ROXs42b, filenames_42b, coords_42b, 1.05, 0.2)
a2 = shift(ROXs12, filenames_12, coords_12, -0.11, -0.25)
    

In [ ]:
#Part 3

#separate shift into an integer and fraction shift

#we are going to match up to 0,0.
from scipy.ndimage.interpolation import shift

#txtfiles = np.genfromtxt(ROXs42b + 'txtfiles.txt', dtype='str')
image_array = []
print(txtfiles)

def imshift(directory):
    txtfiles = np.genfromtxt(directory + 'txtfiles.txt', dtype='str')
    image_array = []
    
    for t in txtfiles:
        star_x, star_y = np.genfromtxt(directory + t)[1:3]
        file = np.genfromtxt(directory + t, dtype=np.str)[0]
        print(file)
        fitsfile = fits.open(directory + str(file))
        image = fitsfile[0].data
        #header = fitsfile.header

        delx = np.diff([star_x, 512])[0]
        dely = np.diff([star_y, 512])[0]
        intx = np.fix(delx)
        inty = np.fix(dely)
        fracx = delx - intx
        fracy = dely - inty

        if fracx < 0.:
            fracx = fracx + 1
            intx = intx - 1
        if fracy < 0.:
            fracy = fracy + 1
            inty = inty - 1

        x = shift(image,[inty,intx])
        image_array.append(x)

        newfilename = file.split("_")[0]

        hdu = fits.PrimaryHDU(x)
        hdulist = fits.HDUList([hdu])
        hdulist.writeto(directory + str(newfilename)+'_shift.fits', overwrite=True)
        
    sum_psf = np.sum(image_array, axis=0)
    median_psf = np.median(image_array, axis=0)

    hdu_sum = fits.PrimaryHDU(sum_psf)
    hdulist_sum = fits.HDUList([hdu_sum])
    hdulist_sum.writeto(directory+'42bsum.fits', overwrite=True)

    hdu_med = fits.PrimaryHDU(median_psf)
    hdulist_med = fits.HDUList([hdu_med])
    hdulist_med.writeto(directory+'42bmedian.fits', overwrite=True)

    #use bi-linear interpolation between four pixels
    #else:
        #return x * ((1-fracx)*(1-fracy)) +  shift(x,0,1) * ((1-fracx)*fracy)\\
    #+ shift(x,1,0) * (fracx*(1-fracy)) + shift(x,1,1) * fracx*fracy

#b1 = imshift(ROXs42b)
b2 = imshift(ROXs12)


In [ ]:
#Part 4

#We will include rotation with the shift so the star lines up precisely in each image.
from scipy.ndimage.interpolation import rotate

txtfiles = np.genfromtxt(ROXs42b + 'txtfiles.txt', dtype='str')
counter = 0

def imrotshift(directory):
    txtfiles = np.genfromtxt(directory + 'txtfiles.txt', dtype='str')
    image_rot = []
    for t in txtfiles:
        star_x, star_y = np.genfromtxt(directory + t)[1:3]
        file_2 = np.genfromtxt(directory + t, dtype=np.str)[0]
        fitsfile = fits.open(directory + file_2)[0]
        image = fitsfile.data
        header = fitsfile.header

        delx = np.diff([star_x, 512])[0]
        dely = np.diff([star_y, 512])[0]
        intx = np.fix(delx)
        inty = np.fix(dely)
        fracx = delx - intx
        fracy = dely - inty

        if fracx < 0.:
            fracx = fracx + 1
            intx = intx - 1
        if fracy < 0.:
            fracy = fracy + 1
            inty = inty - 1

        x = shift(image,[inty,intx])

        #now perform a rotation

        #PA = PARANG + ROTPPOSN - EL - INSTANGL

        parang = header['PARANG']
        rotpposn = header['ROTPPOSN']
        el = header['EL']
        instangl = header['INSTANGL']

        angle = parang + rotpposn - el - instangl
        rot = rotate(x, angle, reshape=False)

        image_rot.append(rot)

        newfilename = file_2.split("_")[0]

        hdu_r = fits.PrimaryHDU(rot)#, header=header)
        hdulist_r = fits.HDUList([hdu_r])
        hdulist_r.writeto(directory  + str(newfilename)+'_shiftrot.fits', overwrite=True)
    
    sum_rotpsf = np.sum(image_rot, axis=0)
    median_rotpsf = np.median(image_rot, axis=0)

    hdu_sum = fits.PrimaryHDU(sum_rotpsf)
    hdulist_sum = fits.HDUList([hdu_sum])
    hdulist_sum.writeto(directory +'42brotsum.fits', overwrite=True)

    hdu_med = fits.PrimaryHDU(median_rotpsf)
    hdulist_med = fits.HDUList([hdu_med])
    hdulist_med.writeto(directory +'42brotmedian.fits', overwrite=True)
    
c1 = imrotshift(ROXs42b)
c2 = imrotshift(ROXs12)

In [ ]:
#Part 5

#Calculate the brightness profile for each star and subtract the median.

def bp(directory):
    image_array = []
    shifted_files = np.genfromtxt(directory + 'paramfile3.dat', dtype='str')
    for sf in shifted_files:
        image = fits.open(directory + sf)[0].data
        newfilename = sf.split("_")[0]
        central_pixel = np.array([512, 512])
        region_1 = 5

        while region_1 < 150:
            flux_array = np.zeros((1024, 1024))
            for i, row in enumerate(image[512-region_1:512+region_1], 512-region_1):
                for j, val in enumerate(row[512-region_1:512+region_1], 512-region_1):
                    distance = np.sqrt(np.diff([i, 512])**2 + np.diff([j, 512])**2)
                    if distance < region_1 and distance > region_1 - 5:
                        flux_array[i, j] = val
            fluxshape = flux_array.shape
            fluxflat = flux_array.flatten()
            med = np.median(fluxflat[np.where(fluxflat>0)])
            for f in range(0, len(fluxflat)):
                if fluxflat[f] != 0:
                    fluxflat[f] = med

            flux_array = fluxflat.reshape(fluxshape)
            image = image - flux_array

            region_1 += 5

        image_array.append(image)     
        hdu = fits.PrimaryHDU(image)
        hdulist = fits.HDUList([hdu])
        hdulist.writeto(directory + str(newfilename) + 'bp.fits', overwrite=True)

    sum_bppsf = np.sum(image_array, axis=0)
    median_bppsf = np.median(image_array, axis=0)

    hdu_sum = fits.PrimaryHDU(sum_bppsf)
    hdulist_sum = fits.HDUList([hdu_sum])
    hdulist_sum.writeto(directory+'bpsum.fits', overwrite=True)

    hdu_med = fits.PrimaryHDU(median_bppsf)
    hdulist_med = fits.HDUList([hdu_med])
    hdulist_med.writeto(directory+'bpmedian.fits', overwrite=True)
    
d1 = bp(ROXs42b)
#d2 = bp(ROXs12)



In [ ]:
# Part 6

def calib(directory):
    shifted_files = np.genfromtxt(directory + 'paramfile3.dat', dtype='str')
    calibration = fits.open(directory+'median.fits')[0].data
    image_array = []
    for sf in shifted_files:
        image = fits.open(directory + sf)[0].data
        newfilename = sf.split("_")[0]
        central_pixel = 512
        radius = 30

        flux_ratios = []

        for i, row in enumerate(image[central_pixel-radius:central_pixel+radius], central_pixel-radius):
            for j, val in enumerate(row[central_pixel-radius:central_pixel+radius], central_pixel-radius):
                fr = val / calibration[i, j]
                if np.isnan(fr) == True:
                    fr = 0
                elif np.isinf(fr) == True:
                    fr = 0
                flux_ratios.append(fr)

        flux_ratios = np.asarray(flux_ratios)
        median = np.median(flux_ratios[np.where(flux_ratios>0)])

        if np.isnan(median) == True:
            calibration = calibration
        else:
            calibration = median * calibration

        image = image - calibration
        image_array.append(image)
        hdu_medsub = fits.PrimaryHDU(image)
        hdulist_medsub = fits.HDUList([hdu_medsub])
        hdulist_medsub.writeto(directory+ str(newfilename) + '_mediansubtracted.fits', overwrite=True)

    sum_mspsf = np.sum(image_array, axis=0)
    median_mspsf = np.median(image_array, axis=0)

    hdu_sum = fits.PrimaryHDU(sum_mspsf)
    hdulist_sum = fits.HDUList([hdu_sum])
    hdulist_sum.writeto(directory+'mssum.fits', overwrite=True)

    hdu_med = fits.PrimaryHDU(median_mspsf)
    hdulist_med = fits.HDUList([hdu_med])
    hdulist_med.writeto(directory+'msmedian.fits', overwrite=True)
    
e1 = calib(ROXs42b)
e2 = calib(ROXs12)

In [ ]:
#Part 7

filenames_42b = np.genfromtxt(ROXs42b + 'paramfile3.dat', dtype='str')
filenames_12 = np.genfromtxt(ROXs12 + 'paramfile3.dat', dtype='str')

def matching(directory1, directory2, filelist1, filelist2, option=None):
    matches = []
    image_array = []
    for f in filelist1:
        image1 = fits.open(directory1 + f)[0].data
        chi_square_array = []
        best_fit_image = []

        for F in filelist2:
            image2 = fits.open(directory2 + F)[0].data
            flux_ratios = []
            for i, row in enumerate(image2):
                for j, val in enumerate(row):
                    flux_ratios.append(val / image1[i, j])

            median = np.median(flux_ratios)

            image2 = median * image2

            chi_square = (image1-image2)**2 / abs(image2)
            total_chi_square = np.sum(chi_square)
            chi_square_array.append(total_chi_square)
            best_fit_image.append(F)

        chi_min = min(chi_square_array)
        best_fit_location = np.where(chi_square_array == chi_min)[0]
        best_image = best_fit_image[best_fit_location[0]]
        image1 = image1 - fits.open(directory2 + best_image)[0].data
        image_array.append(image1)

        hdu = fits.PrimaryHDU(image1)
        hdulist = fits.HDUList([hdu])
        
        if option==None:
            hdulist.writeto(f + '_matchwROXS12_' + best_image + '.fits', overwrite=True)
        else:
            hdulist.writeto(f + '_matchwROXS42b_' + best_image + '.fits', overwrite=True)

        matches.append([f, best_image])
        print(min(chi_square_array))

    np.savetxt(directory1+'matches.txt', matches, fmt='%s')

    sum_chipsf = np.sum(image_array, axis=0)
    median_chipsf = np.median(image_array, axis=0)

    hdu_sum = fits.PrimaryHDU(sum_chipsf)
    hdulist_sum = fits.HDUList([hdu_sum])
    hdulist_sum.writeto(directory1+'chisum.fits', overwrite=True)

    hdu_med = fits.PrimaryHDU(median_chipsf)
    hdulist_med = fits.HDUList([hdu_med])
    hdulist_med.writeto(directory1+'chimedian.fits', overwrite=True)
    
f1 = matching(ROXs42b, ROXs12, filenames_42b, filenames_12)
f2 = matching(ROXs12, ROXs42b, filenames_12, filenames_42b, 1)

In [ ]:
#Part 8
from astropy import units as u

dist_to_42b = (440.3 * u.lightyear).to('AU').value
dist_to_12 = (120 * u.pc).to('AU').value

x = 612.533
y = 470.699

comp42bx = 664.788
comp42by = 642.393

comp12x = 672.252
comp12y = 639.904

dist_comp_42b = np.sqrt((comp42bx - x)**2 + (comp42by - y)**2)
dist_comp_12 = np.sqrt((comp12x - x)**2 + (comp12y - y)**2)

theta_42b = (np.pi/2 - np.arctan(comp42by/comp42bx)) * 180./np.pi
theta_12 = (np.pi/2 - np.arctan(comp12y/comp12x)) * 180./np.pi

angsep_42b = dist_comp_42b * 2.76444e-06 * np.pi/180. * 1./3600. #radians
angsep_12 = dist_comp_12 * 2.764446e-06 * np.pi/180. * 1./3600.

r_42b = angsep_42b * dist_to_42b 
r_12 = angsep_12 * dist_to_12

print(dist_to_42b, dist_comp_42b, theta_42b, angsep_42b, r_42b)
print(dist_to_12, dist_comp_12, theta_12, angsep_12, r_12)